## Prétraitement des référentiels de Pôle Emploi

Pôle emploi maintient les nomenclatures ROME et OGR des métiers et l'INSEE maintient la nomenclature NAF des secteurs d'activité.

Ces nomenclatures sont publiées sous forme de documents bureautiques (XLSX) qui ne sont pas directement utilisables programmatiquement.

* J'ai converti la nomenclature NAF en format CSV directement avec le tableur LibreOffice calc. Les codes sont publiés avec un point qui sépare les 2 premiers chiffres des 2 derniers. J'ai créé une colonne des codes NAF sans le point. Ce notebook ne fait que renommer le header.

* La nomenclature OGR fournie par Pôle emploi réunie dans un même document XLSX les codes OGR, ROME ainsi que les familles et domaines de métiers. J'ai séparé ces informations en différents CSV par des opérations de tri avec LibreOffice calc puis avec ce notebook.


In [1]:
import pandas as pd
from slugify import slugify

## Charge les labels ROME

In [2]:
familles = pd.read_csv('referentiel_ROME/famille_2015_09.csv', sep='|', encoding="utf-8", header=None, dtype=str, index_col=0)
domaines = pd.read_csv('referentiel_ROME/domaine_2015_09.csv', sep='|', encoding="utf-8", header=None, dtype=str)

In [3]:
ROME_df = pd.read_csv('referentiel_ROME/20150921_arboprincipale28427_ROME_tmp.csv', sep='|', encoding="utf-8", header=None, dtype=str)

In [4]:
ROME_df.columns = ['famille', 'domaine', 'metier', 'label']
ROME_df['ROME'] = ROME_df['famille'] + ROME_df['domaine'] + ROME_df['metier']
ROME_df.set_index('ROME', inplace=True)

In [5]:
ROME_df['slugs'] = [slugify(r) for r in ROME_df.label]

In [6]:
ROME_df['famille_label'] = ROME_df.famille.map(lambda code: familles.loc[code][1])

In [7]:
def find_domaine(famille, domaine):
    famille_match = domaines[domaines[0]==famille]
    domaine_match = famille_match[famille_match[1]==domaine]
    return domaine_match.iloc[0][2]

ROME_df['domaine_label'] = ROME_df.apply(lambda row: find_domaine(row.famille, row.domaine), axis=1)

In [9]:
ROME_df.to_csv('referentiel_ROME/20150921_arboprincipale28427_ROME.csv', sep='|')

In [10]:
ROME_df = pd.read_csv('referentiel_ROME/20150921_arboprincipale28427_ROME.csv', index_col=0, sep='|', dtype=str)
ROME_df

,famille,domaine,metier,label,slugs,famille_label,domaine_label
ROME,,,,,,,
A1101,A,11,01,Conduite d'engins agricoles et forestiers,conduite-d-engins-agricoles-et-forestiers,"AGRICULTURE ET PÊCHE, ESPACES NATURELS ET ESPA...",Engins agricoles et forestiers
A1201,A,12,01,Bûcheronnage et élagage,bucheronnage-et-elagage,"AGRICULTURE ET PÊCHE, ESPACES NATURELS ET ESPA...",Espaces naturels et espaces verts
A1202,A,12,02,Entretien des espaces naturels,entretien-des-espaces-naturels,"AGRICULTURE ET PÊCHE, ESPACES NATURELS ET ESPA...",Espaces naturels et espaces verts
A1203,A,12,03,Entretien des espaces verts,entretien-des-espaces-verts,"AGRICULTURE ET PÊCHE, ESPACES NATURELS ET ESPA...",Espaces naturels et espaces verts
A1204,A,12,04,Protection du patrimoine naturel,protection-du-patrimoine-naturel,"AGRICULTURE ET PÊCHE, ESPACES NATURELS ET ESPA...",Espaces naturels et espaces verts
A1205,A,12,05,Sylviculture,sylviculture,"AGRICULTURE ET PÊCHE, ESPACES NATURELS ET ESPA...",Espaces naturels et espaces verts
A1301,A,13,01,Conseil et assistance technique en agriculture,conseil-et-assistance-technique-en-agriculture,"AGRICULTURE ET PÊCHE, ESPACES NATURELS ET ESPA...",Etudes et assistance technique
A1302,A,13,02,Contrôle et diagnostic technique en agriculture,controle-et-diagnostic-technique-en-agriculture,"AGRICULTURE ET PÊCHE, ESPACES NATURELS ET ESPA...",Etudes et assistance technique
A1303,A,13,03,Ingénierie en agriculture et environnement nat...,ingenierie-en-agriculture-et-environnement-nat...,"AGRICULTURE ET PÊCHE, ESPACES NATURELS ET ESPA...",Etudes et assistance technique


In [11]:
ROME_df.iloc[5].label, ROME_df.loc['A1205'].label, ROME_df.index.get_loc('A1205'), ROME_df.iloc[5].name

('Sylviculture', 'Sylviculture', 5, 'A1205')

In [12]:
len(ROME_df)

531

In [13]:
for ROME_code in ROME_df.index:
    assert '"' not in ROME_df.loc[ROME_code].label

## Métiers

In [ ]:
OGR_df = pd.read_csv('referentiel_ROME/20150921_arboprincipale28427_OGR_tmp.csv', sep='|', encoding="utf-8", header=None, dtype=str)
OGR_df.columns = ['ROME1', 'ROME2', 'ROME3', 'label', 'OGR']
OGR_df['ROME'] = OGR_df['ROME1'] + OGR_df['ROME2'] + OGR_df['ROME3']
OGR_df.set_index('OGR', inplace=True)
OGR_df['slugs'] = [slugify(r) for r in OGR_df.label]
OGR_df.to_csv('referentiel_OGR/20150921_arboprincipale28427_OGR.csv', sep='|')

In [ ]:
OGR_df = pd.read_csv('referentiel_OGR/20150921_arboprincipale28427_OGR.csv', sep='|', dtype=str).set_index('OGR')
OGR_df

# Codes NAF

In [ ]:
NAF_df = pd.read_csv('referentiel_NAF/naf2008_liste_n5.csv', sep='|', encoding="utf-8").set_index(['Code2'])
NAF_df.index.names = ['NAF']
NAF_df.columns = ['NAF_alternatif', 'label']
NAF_df.to_csv('referentiel_NAF/naf2008_liste_n5_nouveau_header.csv', sep='|')

In [ ]:
NAF_df = pd.read_csv('referentiel_NAF/naf2008_liste_n5_nouveau_header.csv', sep='|', encoding="utf-8").set_index(['NAF'])
NAF_df

In [ ]:
for NAF_code in NAF_df.index:
    assert '"' not in NAF_df.loc[NAF_code].label